#RNN

In this notebook I tried to solve the classification model with an RNN.

## TODO:
- [ ] Stemming
- [x] Remove Stop Words


In [1]:
#check if tensorflow is running on GPU
import tensorflow as tf 

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("./dataset.csv")
df

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False
...,...,...
199995,Conor maynard seamlessly fits old-school r&b h...,False
199996,How to you make holy water? you boil the hell ...,True
199997,How many optometrists does it take to screw in...,True
199998,Mcdonald's will officially kick off all-day br...,False


In [4]:
df.describe()

,text,humor
count,200000,200000
unique,200000,2
top,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
freq,1,100000


# Check for imbalance

The Dataset is equally balance, no need for a rebalance

In [5]:
df["humor"].value_counts()

False    100000
True     100000
Name: humor, dtype: int64

# Preprocessing

In [6]:
# Transform class from Boolean to integer value
df['label']=df['humor'].apply(lambda x: 1 if x==True else 0)

In [7]:
#remove stop words
from gensim.parsing.preprocessing import remove_stopwords
df["text"] = df["text"].apply(lambda x: remove_stopwords(x))

In [8]:
import tensorflow as tf

all_dataset = ( 
    tf.data.Dataset.from_tensor_slices(
                        ( tf.cast(df["text"].values, tf.string),
                         tf.cast(df["label"].values, tf.int32)
                        
                        )

)
                   
                   )

In [9]:
#for features_tensor, target_tensor in training_dataset:
#    print(f'features:{features_tensor} target:{target_tensor}')
# 10% from 200 000 is 20 000, the rest goes for training.
test_dataset = all_dataset.take(20000)
train_dataset = all_dataset.skip(20000)

#for features_tensor, target_tensor in train_dataset:
#    print(f'features:{features_tensor} target:{target_tensor}')

In [10]:
#we could shuffle the data here.
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [11]:
encoder = tf.keras.layers.TextVectorization()
encoder.adapt(train_dataset.map(lambda text, label: text))

In [12]:
import numpy as np

In [13]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc3 in position 7: unexpected end of data

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
print([layer.supports_masking for layer in model.layers])

In [ ]:
# predict on a sample text without padding.

sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

In [ ]:
# predict on a sample text with padding

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)